In [3]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load FER2013 CSV
data = pd.read_csv(r'C:\Users\ASUS\Desktop\emotion detection 1\fer2013\fer2013.csv')

# Extract data
pixels = data['pixels'].tolist()
faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split()]
    face = np.asarray(face).reshape(48, 48)
    face = face.astype('float32') / 255.0  # normalize
    faces.append(face)

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)  # shape: (n, 48, 48, 1)

emotions = pd.get_dummies(data['emotion']).values  # one-hot encode

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(faces, emotions, test_size=0.2, random_state=42)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(48,48,1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotions
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 21, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     6,554,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,632,967 (25.30 MB)

 Trainable params: 6,632,583 (25.30 MB)

 Non-trainable params: 384 (1.50 KB)

In [5]:
model.fit(X_train, y_train, batch_size=64, epochs=30, validation_data=(X_val, y_val))


Epoch 1/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 75s 163ms/step - accuracy: 0.2461 - loss: 3.4009 - val_accuracy: 0.2522 - val_loss: 5.4205
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 75s 167ms/step - accuracy: 0.3360 - loss: 1.6454 - val_accuracy: 0.3079 - val_loss: 1.8252
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 74s 165ms/step - accuracy: 0.3772 - loss: 1.5587 - val_accuracy: 0.4028 - val_loss: 1.5132
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 80s 178ms/step - accuracy: 0.3966 - loss: 1.5221 - val_accuracy: 0.4074 - val_loss: 1.5278
Epoch 5/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 87s 194ms/step - accuracy: 0.4182 - loss: 1.4874 - val_accuracy: 0.4401 - val_loss: 1.4311
Epoch 6/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 103s 230ms/step - accuracy: 0.4339 - loss: 1.4503 - val_accuracy: 0.4595 - val_loss: 1.4010
Epoch 7/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 110s 244ms/step - accuracy: 0.4451 - loss: 1.4232 - val_accuracy: 0.4501 - val_loss: 1.4258
Epoch 8/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 110s 246ms/step - accuracy: 0.4671 - loss

In [6]:
model.save("emotion_model.h5")


In [7]:
import os
print("Saved:", os.path.exists("emotion_model.h5"))


Saved: True
